# Python 実践データ分析 100本ノック 第2版

(Python Practical Data Analysis 100 Knocks, 2nd ed.)

### 第5部 放課後練：さらなる挑戦

(Part 5: After-School Practice: Further Challenges)

### 第12章 データ加工に挑戦する10本ノック

 (Chapter 12: 10 knocks to challenge data processing)

In [2]:
# Connect with Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Move to working folder
import os
os.chdir('/content/drive/MyDrive/Colab-Notebooks/python-100knock-data-analytics/chap12/')

### knock 111 : Common “Excel Data Challenges”

Handling cell-joined data

In [1]:
import pandas as pd

In [6]:
input_data = pd.read_excel("12-1.xlsx")
input_data.head()

,都道府県,市区町村,人数（男性、女性）
0,東京都,新宿区,12
1,NaN,NaN,14
2,NaN,豊島区,15
3,NaN,NaN,13
4,神奈川県,横浜市,8


In [7]:
# Organize data by having headcount items horizontally for each gender.
men = input_data['人数（男性、女性）'][0::2]
women = input_data['人数（男性、女性）'][1::2]

men.reset_index(inplace=True, drop=True)
women.reset_index(inplace=True, drop=True)

output_data = input_data[0::2].copy()
output_data.reset_index(inplace=True, drop=True)
output_data['男性'] = men
output_data['女性'] = women

# Delete the column “Number of persons (male, female)” which is no longer needed.
output_data.drop('人数（男性、女性）', axis=1, inplace=True)

# Set missing prefectures
output_data.iat[1, 0] = output_data.iat[0, 0]
output_data.iat[3, 0] = output_data.iat[2, 0]

output_data.head()

,都道府県,市区町村,男性,女性
0,東京都,新宿区,12,14
1,東京都,豊島区,15,13
2,神奈川県,横浜市,8,9
3,神奈川県,横須賀市,5,2


In [8]:
output_data.to_csv('12-1_out.csv', index=False)

### knock 112 : Excel employee master processing challenge

In [9]:
input_data = pd.read_excel("12-2.xlsx")
input_data

,社員名,生年月日,部署,役職,更新日
0,田中 正,1975-10-09,A部,課長,2021-10-12
1,水野 メイサ,1981-02-23,C部,課長,2020-05-08
2,齊藤 隆,2001-01-23,B部,NaN,2021-05-08
3,茂木 新人,2002-08-23,A部,NaN,2021-04-01
4,篠山 雅功,1992-07-02,A部,課長,2022-03-02
5,水野 メイサ,1981-02-23,C部,部長,2021-12-08
6,白鳥 りえ,1999-04-11,B部,NaN,2022-02-01
7,田中 正,1975-10-09,A部,部長,2022-03-02
8,篠山 雅功,1992-07-02,B部,NaN,2021-01-15


In [10]:
# Make the NaN of the position a blank character.
output_data = input_data.copy()
output_data['役職'].fillna('', inplace=True)

# Sort data by update date
output_data.sort_values('更新日', ascending=True, inplace=True)

# Identify duplicate data and retain new data
output_data.drop_duplicates(subset=['社員名', '生年月日'], keep='last', inplace=True)

output_data

<ipython-input-10-74dd35fde8d3>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  output_data['役職'].fillna('', inplace=True)


,社員名,生年月日,部署,役職,更新日
3,茂木 新人,2002-08-23,A部,,2021-04-01
2,齊藤 隆,2001-01-23,B部,,2021-05-08
5,水野 メイサ,1981-02-23,C部,部長,2021-12-08
6,白鳥 りえ,1999-04-11,B部,,2022-02-01
4,篠山 雅功,1992-07-02,A部,課長,2022-03-02
7,田中 正,1975-10-09,A部,部長,2022-03-02


In [11]:
output_data.to_csv('12-2_out.csv', index=False)

### knock 113 : Normalization challenge

In [12]:
input_data = pd.read_excel("12-3.xlsx")
input_data

,仕入先,仕入先TEL,商品,販売単価,入荷日
0,A農家,03-4444-4444,きゃべつ,100,2019-07-01
1,NaN,NaN,れたす,80,2019-07-03
2,NaN,NaN,きゃべつ,100,2019-08-01
3,B農家,042-222-3333,もやし,20,2019-07-08


In [13]:
# Fill in the missing values of the cell join (in this case, only the A farmer is missing, so fillna can be used).
input_data['仕入先'].fillna(input_data['仕入先'][0], inplace=True)
input_data['仕入先TEL'].fillna(input_data['仕入先TEL'][0], inplace=True)

# Stores suppliers in a separate data frame
farmer_data = input_data[['仕入先', '仕入先TEL']].copy()
farmer_data.drop_duplicates(inplace=True)

# Generate supplier key
farmer_idx = []
idx = 0
for item in farmer_data['仕入先']:
  idx += 1
  farmer_idx.append('F' + str(idx))

# Granted key
farmer_data.insert(0, '仕入先ID', farmer_idx)

# Stores product data in a separate data frame
product_data = input_data[['商品','販売単価']].copy()
product_data.drop_duplicates(inplace=True)

# Generate product key
product_idx = []
idx = 0
for item in product_data['商品']:
  idx += 1
  product_idx.append('P' + str(idx))

# Granted key
product_data.insert(0, '商品ID', product_idx)

# Convert transaction table to key
order_data = pd.merge(input_data, farmer_data[['仕入先ID', '仕入先']], how='left', on='仕入先')
order_data = pd.merge(order_data, product_data[['商品ID', '商品']], how='left', on='商品')
order_data = order_data[['仕入先ID','商品ID', '入荷日']]
order_data

<ipython-input-13-1af3e25490a5>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  input_data['仕入先'].fillna(input_data['仕入先'][0], inplace=True)
<ipython-input-13-1af3e25490a5>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True

,仕入先ID,商品ID,入荷日
0,F1,P1,2019-07-01
1,F1,P2,2019-07-03
2,F1,P1,2019-08-01
3,F2,P3,2019-07-08


In [14]:
farmer_data

,仕入先ID,仕入先,仕入先TEL
0,F1,A農家,03-4444-4444
3,F2,B農家,042-222-3333


In [15]:
product_data

,商品ID,商品,販売単価
0,P1,きゃべつ,100
1,P2,れたす,80
3,P3,もやし,20


In [16]:
order_data.to_csv('12-3_order.csv', index=False)
farmer_data.to_csv('12-3_farmer.csv', index=False)
product_data.to_csv('12-3_product.csv', index=False)

### knock 114 : Challenges in processing outliers

In [17]:
input_data = pd.read_csv("12-4.csv")
input_data['金額'].describe().astype('int')

,金額
count,200
mean,9299
std,76251
min,363
25%,1331
50%,2541
75%,4840
max,1076449


In [18]:
# Get 3rd quartile
threshold = input_data['金額'].quantile(0.75)

# Replace data beyond the third quartile with the third quartile
output_data = input_data.copy()
output_data.loc[input_data['金額']>threshold, '金額'] = threshold
output_data['金額'].describe()

,金額
count,200.00000
mean,2896.34500
std,1486.26091
min,363.00000
25%,1331.00000
50%,2541.00000
75%,4840.00000
max,4840.00000


In [19]:
output_data['金額'].describe().to_csv('12-4_out.csv')

### knock 115 : Challenging to complete missing values

In [20]:
input_data = pd.read_excel("12-5.xlsx")
input_data

,顧客名,都道府県,市区町村,年齢
0,須賀ひとみ,東京,H市,20.0
1,岡田 敏也,神奈川,E市,23.0
2,芳賀 希,東京,A市,44.0
3,荻野 愛,神奈川,F市,21.0
4,栗田 憲一,神奈川,E市,49.0
5,梅沢 麻緒,東京,A市,18.0
6,相原 ひとり,東京,H市,NaN
7,新村 丈史,埼玉,B市,29.0
8,石川 まさみ,NaN,G市,33.0
9,小栗 正義,埼玉,G市,87.0


In [21]:
import math

In [22]:
output_data = input_data.copy()

# Complete missing data for prefectures
target_div = output_data.loc[output_data['都道府県'].isnull(), '市区町村']
for division in target_div:
  output_data.loc[(output_data['都道府県'].isnull()) & (output_data['市区町村']==division), '都道府県'] = output_data.loc[(output_data['市区町村']==division) & ~(output_data['都道府県'].isnull()), '都道府県'].unique()[0]

# Complete missing age data
target_div = output_data.loc[output_data['年齢'].isnull(), '市区町村']
for division in target_div:
  output_data.loc[(output_data['年齢'].isnull()) & (output_data['市区町村']==division), '年齢'] = math.floor(output_data.loc[output_data['市区町村']==division, '年齢'].mean())

output_data

,顧客名,都道府県,市区町村,年齢
0,須賀ひとみ,東京,H市,20.0
1,岡田 敏也,神奈川,E市,23.0
2,芳賀 希,東京,A市,44.0
3,荻野 愛,神奈川,F市,21.0
4,栗田 憲一,神奈川,E市,49.0
5,梅沢 麻緒,東京,A市,18.0
6,相原 ひとり,東京,H市,52.0
7,新村 丈史,埼玉,B市,29.0
8,石川 まさみ,埼玉,G市,33.0
9,小栗 正義,埼玉,G市,87.0


In [23]:
output_data.to_csv('12-5_out.csv', index=False)

### knock 116 :  Challenge to scramble data

In [24]:
input_data = pd.read_excel("12-6.xlsx")
input_data.head()

,氏名,購入金額
0,須賀ひとみ,2131
1,須賀ひとみ,5213
2,稲田 将也,3292
3,西脇 礼子,1122
4,栗田 憲一,4823


In [25]:
# Show pre-scrambled totals for verification
input_data.groupby('氏名').sum()

,購入金額
氏名,
内村 まさみ,5712
小口 豊,9452
岩佐 孝太郎,5818
栗田 憲一,4823
相原 ひとり,1022
稲田 将也,29026
荻野 愛,24506
西脇 礼子,23149
須賀ひとみ,8896


In [26]:
import hashlib

In [27]:
output_data = input_data.copy()

# Name hashed
output_data['氏名'] = output_data['氏名'].apply(lambda x: hashlib.sha256(x.encode()).hexdigest())
# Aggregate by hashed name
output_data.groupby('氏名').sum()

,購入金額
氏名,
061027bd9eb2c1109262ceb6b7067cd71a1811ecc2844d828acacf5555a738f9,5818
0d60d878da37366d7d67f103569520b03a6fcde49a2963a1997bea5b6b9b7b40,4823
0f8d4726fc84296eef6d8a9dbf631cb978caecd759a1582215200f98d8b82e9a,8896
15416e267a8fb81d36a34c02f84d3efefeaac0f5d085a7446555bba32f42e6ba,29026
1a6c9503a1f518faec9891fed43e0228b1d76404486a1d904b73b6b8ef8ea032,23149
51659db65801946f4a3e3f234eb74f240dd2181ee0da7fbe4a44c540835d206c,5723
5a33912e5b3a3fa4f7b53018bfbc1ef798a552bf06b7c1aad023029153d194f6,24506
7738026cec1b844dcfa4b4f3f1fc7d5f701a1d0d8a2a324393c237242035c395,5712
a92e8099c51126ed4fca5aaf2af7c75f2b0cafddc4c5f0212003f90730a861aa,9452


In [28]:
output_data.groupby('氏名').sum().to_csv('12-6_out.csv')

### knock 117 : Challenging automatic character code determination

In [29]:
import chardet

In [30]:
# Check the character encoding of a file

files = ['12-7-1.csv', '12-7-2.csv', '12-7-3.csv']
df = pd.DataFrame()

for file in files:
  # Open file in binary
  with open(file, mode='rb') as f:
    contents = f.read()
    enc = chardet.detect(contents)['encoding']
    # Read the file with the character encoding found.
    df = pd.concat([df, pd.read_csv(file, encoding=enc)])

df

,file,text
0,12-7-1,このファイルはSJISで記載されています。
0,12-7-2,このファイルはUTF-16で記載されています。
0,12-7-3,このファイルはEUC-JPで記載されています。


In [31]:
df.to_csv('12-7_out.csv', index=False)

### knock 118 : Challenges in processing sensor data

In [32]:
sensor1 = pd.read_csv('12-8-1.csv', index_col=0)
sensor2 = pd.read_csv('12-8-2.csv', index_col=0)

display(sensor1.head())
display(sensor2.head())

,sensor_1,sensor_2
time_stamp,,
2020/12/17 19:41:05.411,-1485,0
2020/12/17 19:41:05.596,-1817,0
2020/12/17 19:41:05.795,-1863,0
2020/12/17 19:41:05.996,-1871,0
2020/12/17 19:41:06.199,-1931,0


,sensor_3,sensor_4
time_stamp,,
2020/12/17 19:41:05.519,0,-1201
2020/12/17 19:41:05.703,0,-1536
2020/12/17 19:41:05.905,0,-1624
2020/12/17 19:41:05.098,0,-1638
2020/12/17 19:41:06.307,0,-1641


In [33]:
# Combine two sensor values and sort by time stamp
df_main = pd.concat([sensor1, sensor2], ignore_index=False)
df_main.sort_values('time_stamp', inplace=True)

# Linear interpolation of missing values
df_main.interpolate(method='linear', inplace=True)

# The first line cannot be interpolated, so it is set to 0.
df_main.iat[0,0] = 0
df_main.iat[0,1] = 0

df_main

,sensor_1,sensor_2,sensor_3,sensor_4
time_stamp,,,,
2020/12/17 19:41:05.098,0.0,0.0,0.000000,-1638.0
2020/12/17 19:41:05.411,-1485.0,0.0,0.000000,-1419.5
2020/12/17 19:41:05.519,-1651.0,0.0,0.000000,-1201.0
2020/12/17 19:41:05.596,-1817.0,0.0,0.000000,-1368.5
2020/12/17 19:41:05.703,-1840.0,0.0,0.000000,-1536.0
...,...,...,...,...
2020/12/17 19:41:31.998,-1960.0,0.0,-1813.666667,0.0
2020/12/17 19:41:32.199,-1373.0,0.0,-1877.333333,0.0
2020/12/17 19:41:32.307,-845.5,0.0,-1941.000000,0.0


In [34]:
df_main.to_csv('12-8_out.csv')

### knock 119 : Challenge JSON format

In [35]:
input_data = pd.read_csv("12-9.csv")
input_data

,order_id,customer_id,order_accept_date,total_amount
0,79339111,C26387220,2022-04-01 11:00:00,4144
1,18941733,C48773811,2022-04-01 11:00:00,2877
2,56217880,C24617924,2022-04-01 11:00:00,2603
3,28447783,C26387220,2022-04-01 11:00:00,2732
4,32576156,C54568117,2022-04-01 11:00:00,2987
5,75629806,C38583902,2022-04-30 21:57:57,3050
6,91002809,C48773811,2022-04-30 21:57:57,4692
7,3021273,C24617924,2022-04-30 21:57:57,2388
8,82302078,C26387220,2022-04-30 21:57:57,2603
9,97601615,C54568117,2022-04-30 21:57:57,1899


In [36]:
import json

In [44]:
# Converted to JSON and output to file
input_data.to_json('12-9_out.json')

In [45]:
# Read JSON format files directly into the data frame
read_json = pd.read_json('12-9_out.json')
read_json.head()

,order_id,customer_id,order_accept_date,total_amount
0,79339111,C26387220,2022-04-01 11:00:00,4144
1,18941733,C48773811,2022-04-01 11:00:00,2877
2,56217880,C24617924,2022-04-01 11:00:00,2603
3,28447783,C26387220,2022-04-01 11:00:00,2732
4,32576156,C54568117,2022-04-01 11:00:00,2987


In [47]:
# Read JSON format files in dictionary format
with open('12-9_out.json') as f:
  dict_json = json.load(f)
dict_json

{'order_id': {'0': 79339111,
  '1': 18941733,
  '2': 56217880,
  '3': 28447783,
  '4': 32576156,
  '5': 75629806,
  '6': 91002809,
  '7': 3021273,
  '8': 82302078,
  '9': 97601615,
  '10': 87676506,
  '11': 65713874,
  '12': 80343997,
  '13': 1798443,
  '14': 96208608,
  '15': 5060741,
  '16': 70509759,
  '17': 89047729,
  '18': 25546308,
  '19': 91693481,
  '20': 57908119,
  '21': 7145625,
  '22': 31301192},
 'customer_id': {'0': 'C26387220',
  '1': 'C48773811',
  '2': 'C24617924',
  '3': 'C26387220',
  '4': 'C54568117',
  '5': 'C38583902',
  '6': 'C48773811',
  '7': 'C24617924',
  '8': 'C26387220',
  '9': 'C54568117',
  '10': 'C54568117',
  '11': 'C48773811',
  '12': 'C27698225',
  '13': 'C26387220',
  '14': 'C48773811',
  '15': 'C24617924',
  '16': 'C24617924',
  '17': 'C26387220',
  '18': 'C24617924',
  '19': 'C48773811',
  '20': 'C26387220',
  '21': 'C48773811',
  '22': 'C48773811'},
 'order_accept_date': {'0': '2022-04-01 11:00:00',
  '1': '2022-04-01 11:00:00',
  '2': '2022-04-0

### knock 120 :  SQLite Challenge

In [48]:
import sqlite3

In [49]:
# Database name (any name.db)
db_name = 'TrialDatabase.db'
# Connect to database (if database does not exist, create database and connect)
con = sqlite3.connect(db_name)
cur = con.cursor()
# If successful, the database defined in db_name exists on the drive as a file

In [50]:
input_data = pd.read_csv("12-9.csv")
input_data.head()

,order_id,customer_id,order_accept_date,total_amount
0,79339111,C26387220,2022-04-01 11:00:00,4144
1,18941733,C48773811,2022-04-01 11:00:00,2877
2,56217880,C24617924,2022-04-01 11:00:00,2603
3,28447783,C26387220,2022-04-01 11:00:00,2732
4,32576156,C54568117,2022-04-01 11:00:00,2987


In [51]:
# Store csv information in a table (table is created in data frame to_sql and data is entered)
input_data.to_sql('t_data', con, if_exists='replace', index=None)

23

In [52]:
# Query the database with an extract query (SELECT clause) and receive the results in a data frame type
sql = 'select * from t_data;'
df = pd.read_sql_query(sql, con)
df

,order_id,customer_id,order_accept_date,total_amount
0,79339111,C26387220,2022-04-01 11:00:00,4144
1,18941733,C48773811,2022-04-01 11:00:00,2877
2,56217880,C24617924,2022-04-01 11:00:00,2603
3,28447783,C26387220,2022-04-01 11:00:00,2732
4,32576156,C54568117,2022-04-01 11:00:00,2987
5,75629806,C38583902,2022-04-30 21:57:57,3050
6,91002809,C48773811,2022-04-30 21:57:57,4692
7,3021273,C24617924,2022-04-30 21:57:57,2388
8,82302078,C26387220,2022-04-30 21:57:57,2603
9,97601615,C54568117,2022-04-30 21:57:57,1899


In [53]:
df.to_csv('12-10_out.csv', index=False)